# Aluguel

## Pega URL


In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Função para extrair os URLs dos links <a> com a classe 'new-card'
def extrair_urls_anuncios(soup):
    # Encontra todos os elementos 'a' com a classe 'new-card'
    links = soup.find_all('a', class_='new-card')
    # Extrai os URLs dos links
    urls = [link.get('href') for link in links]
    return urls

# Função para fazer o scraping de uma página e extrair os URLs dos links
def fazer_scraping_pagina(url):
    # Faz a requisição HTTP
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai os URLs dos links
        urls_anuncios_pagina = extrair_urls_anuncios(soup)
        return urls_anuncios_pagina
    else:
        print('Falha ao acessar a página:', response.status_code)
        return None

# Função para fazer o scraping de várias páginas
def fazer_scraping_varias_paginas(url_base, num_paginas):
    lista_urls_anuncios_total = []
    for pagina in tqdm(range(1, num_paginas + 1), desc="Progresso"):
        url_pagina = f"{url_base}?pagina={pagina}"
        urls_anuncios_pagina = fazer_scraping_pagina(url_pagina)
        if urls_anuncios_pagina:
            lista_urls_anuncios_total.extend(urls_anuncios_pagina)
    return lista_urls_anuncios_total

# URL base do site de imóveis que você quer fazer scraping
url_base = 'https://www.dfimoveis.com.br/aluguel/df/todos/imoveis'

# Número total de páginas que você deseja percorrer
num_paginas = 244 # Ajuste conforme necessário

# Faz o scraping de várias páginas
urls_anuncios_total = fazer_scraping_varias_paginas(url_base, num_paginas)

# Se os URLs foram obtidos com sucesso
if urls_anuncios_total:
    # Cria um DataFrame com os URLs dos anúncios
    df_link_aluguel = pd.DataFrame({'URL': urls_anuncios_total})
    # Mostra o DataFrame
    print(df_link_aluguel)


Progresso: 100%|██████████| 244/244 [02:07<00:00,  1.91it/s]

                                                    URL
0     /imovel/prdio-0-quartos-aluguel-asa-sul-brasil...
1     /imovel/apartamento-2-quartos-aluguel-noroeste...
2     /imovel/sala-0-quartos-aluguel-asa-norte-brasi...
3     /imovel/galpo-0-quartos-aluguel-polo-jk-santa-...
4     /imovel/sala-0-quartos-aluguel-asa-norte-brasi...
...                                                 ...
7303  /imovel/loja-0-quartos-aluguel-ceilandia-sul-c...
7304  /imovel/galpo-0-quartos-aluguel-vicente-pires-...
7305  /imovel/sala-0-quartos-aluguel-taguatinga-cent...
7306  /imovel/kitnet-1-quarto-aluguel-guara-i-guara-...
7307  /imovel/sala-0-quartos-aluguel-asa-sul-brasili...

[7308 rows x 1 columns]


## Raspagem dos anúncios

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Função para extrair as informações de um anúncio a partir de um URL
def extrair_info_anuncio(url):
    # Faz a requisição HTTP para o URL do anúncio
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai as informações do anúncio
        try:
            titulo = soup.find('h1', class_='mb-0 font-weight-600 mobile-fs-1-5').text.strip()
            preco = soup.find('h6', class_='align-text-left text-normal bb-m mr-6').text.split('\n')[2].replace('.','')
            area = soup.find('h6', class_='text-normal mb-0').text.split()[2].replace('.','').replace(',',".")
            descricao = soup.find('p',class_='w-100 pb-3 mb-0 texto-descricao').text
            try:
               area = float(area)
            except:
               area = None
            try:
               preco = float(preco)
            except:
               preco = None
            try:
               preco_metro = float(area)/float(preco)
            except:
               preco_metro = None
  
            info = soup.find_all("h6", class_='mb-0 text-normal')
            imobiliaria = soup.find('h6', class_='pb-0 mb-0').text
            creci = soup.find('div',class_='col-md-8 mt-2 ml-2').text.split('\n')[2].split()[1]
            tipo = soup.find('div', class_='col-lg-6').text.split()[2]

            possibilidades = [
            'Aquecimento Solar', 'Circuito de TV', 'Gerador de Energia', 'Piscina', 'Sala de Ginástica',
            'Varanda', 'Cozinha Espaçosa', 'Ar Condicionado', 'Cobertura Coletiva', 'Guarita',
            'PlayGround', 'Salão de Festas', 'Projeto de Iluminação', 'Pintura Nova', 'Área de Lazer',
            'Interfone', 'Mobiliado', 'Área de Serviço', 'Brinquedoteca', 'Churrasqueira', 'Cozinha com Armários',
            'Despensa', 'Escritório', 'Gás Canalizado', 'Jardim', 'Lavabo', 'Lazer no Pilotis', 'Poço Artesiano',
            'Portão Eletrônico', 'Quadra Esportiva', 'Sala de Jogos', 'Salão de Jogos', 'Salão Gourmet', 'Sauna',
            'SPA', 'Área de Serviço Coberta', 'Piso em Porcelanato', 'Vista Livre', 'Home Theater', 'Aceita Pet', 'Academia'
            ]

            resultados = {possibilidade: 0 for possibilidade in possibilidades}
            checkbox = soup.find('ul', class_='checkboxes')
            checkbox_text = checkbox.text.strip()
            checkbox_list = checkbox_text.split('\n')

            for item in checkbox_list:
              if item in resultados:
                resultados[item] = 1

            dados_dict = {}
            for i in info:
              item  = i.text.split()
              chave = item[0]
              valor = ' '.join(item[1:])
              if chave in dados_dict:
                dados_dict[chave].append(valor)
              else:
                dados_dict[chave] = [valor]
            try:
              quartos = dados_dict['Quartos:'][0]
            except:
              quartos = 0
            try:
              suites = dados_dict['Suítes:'][0]
            except:
              suites = 0
            try:
              garagem = dados_dict['Garagens:'][0]
            except:
              garagem = 0
            try:
              iptu = dados_dict['IPTU'][0]
            except:
              iptu = '0 SemInformacao'
            try:
              condominio = dados_dict['Condomínio'][0]
            except:
              condominio = '0 SemInformacao'
            try:
              regiao = dados_dict['Cidade:'][0]
            except:
              regiao = '0 SemInformacao'
        except AttributeError as e:
            print(e)
            return None
        # Retorna um dicionário com as informações do anúncio
        return {
            'Título': titulo,
            "Valor": preco,
            "area": area,
            'Valor_metro': preco_metro,
            'quartos': quartos,
            'suites': suites,
            'garagem': garagem,
            'iptu': iptu.split()[1].replace('.',''),
            'condominio': condominio.split()[1].replace('.',''),
            'descricao': descricao,
            'regiao': regiao.split('-')[1].strip(),
            'cidade': regiao,
            'imobiliaria': imobiliaria,
            'creci': creci,
            "operacao": 'aluguel',
            **resultados,
            'tipo': tipo
        }
    else:
        print('Falha ao acessar o anúncio:', response.status_code)
        return None

# Lista de URLs dos anúncios
urls_anuncios = [f"https://www.dfimoveis.com.br/{pagina}" for pagina in df_link_aluguel['URL'].values]

# Lista para armazenar as informações dos anúncios
info_anuncios_total = []

# Extrai as informações de cada anúncio
for url_anuncio in tqdm(urls_anuncios, desc='Extraindo informações dos anúncios'):
    info_anuncio = extrair_info_anuncio(url_anuncio)
    if info_anuncio:
        info_anuncios_total.append(info_anuncio)

# Se as informações foram obtidas com sucesso
if info_anuncios_total:
    # Cria um DataFrame com as informações dos anúncios
    df_aluguel = pd.DataFrame(info_anuncios_total)

df_aluguel

Extraindo informações dos anúncios: 100%|██████████| 7308/7308 [41:24<00:00,  2.94it/s]  


,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,descricao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,SCS Quadra 06 Bloco A Lote 194,7000.0,200.0,0.028571,0,0,0,SemInformacao,2000,"6º andar todo reformado, com 200 m², piso em p...",...,0,0,0,0,0,0,0,0,0,Prédio
1,SQNW 310 Bloco B,4900.0,76.0,0.015510,2,1,2,409,791,Excelente apartamento no condomínio Persona Ve...,...,0,1,0,0,0,0,0,0,0,Apartamento
2,SHN Quadra 2,27000.0,615.0,0.022778,0,0,5,SemInformacao,6183,"Asa Norte, SHN Quadra 02, EXECUTIVE OFFICE, Sa...",...,0,0,0,0,0,0,0,0,0,Sala
3,Polo de Desenv Juscelino Kubitschek Trecho 6 C...,52000.0,2000.0,0.038462,0,0,0,SemInformacao,SemInformacao,Excelentes galpões logísticos recém reformados...,...,0,0,0,0,0,0,0,0,0,Galpão
4,SEPN 516 Bloco E,13830.0,147.0,0.010629,0,0,0,333,2389,Clínica médica na Asa Norte com:\rDiversos con...,...,0,0,0,0,0,0,0,0,0,Sala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7303,QNP 32 Conjunto B,1500.0,150.0,0.100000,0,0,0,SemInformacao,SemInformacao,IMOBILIÁRIA ESTRELA DESDE 1970 – ALUGA LOJA FR...,...,0,0,0,0,0,0,0,0,0,Loja
7304,Rua 5 Chacará 100,15000.0,180.0,0.012000,0,0,0,SemInformacao,SemInformacao,.,...,0,0,0,0,0,0,0,0,0,Galpão
7305,Quadra C 7,700.0,30.0,0.042857,0,0,0,SemInformacao,SemInformacao,Ótima Sala comercial em bem localizada em Tagu...,...,0,0,0,0,0,0,0,0,0,Sala
7306,QE 20 Conjunto N,580.0,30.0,0.051724,1,0,0,SemInformacao,SemInformacao,.,...,0,0,0,0,0,0,0,0,0,Kitnet


## Dados Aluguel

In [25]:
df_aluguel.to_csv('aluguel.csv', index=False)

# Venda

## Pega URL

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

# Função para extrair os URLs dos links <a> com a classe 'new-card'
def extrair_urls_anuncios(soup):
    # Encontra todos os elementos 'a' com a classe 'new-card'
    links = soup.find_all('a', class_='new-card')
    # Extrai os URLs dos links
    urls = [link.get('href') for link in links]
    return urls

# Função para fazer o scraping de uma página e extrair os URLs dos links
def fazer_scraping_pagina(url):
    # Faz a requisição HTTP
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai os URLs dos links
        urls_anuncios_pagina = extrair_urls_anuncios(soup)
        return urls_anuncios_pagina
    else:
        print('Falha ao acessar a página:', response.status_code)
        return None

# Função para fazer o scraping de várias páginas
def fazer_scraping_varias_paginas(url_base, num_paginas):
    lista_urls_anuncios_total = []
    for pagina in tqdm(range(1, num_paginas + 1), desc="Progresso"):
        url_pagina = f"{url_base}?pagina={pagina}"
        urls_anuncios_pagina = fazer_scraping_pagina(url_pagina)
        if urls_anuncios_pagina:
            lista_urls_anuncios_total.extend(urls_anuncios_pagina)
    return lista_urls_anuncios_total

# URL base do site de imóveis que você quer fazer scraping
url_base = 'https://www.dfimoveis.com.br/venda/df/todos/imoveis'

# Número total de páginas que você deseja percorrer
num_paginas = 1414 # Ajuste conforme necessário

# Faz o scraping de várias páginas
urls_anuncios_total = fazer_scraping_varias_paginas(url_base, num_paginas)
# Se os URLs foram obtidos com sucesso
if urls_anuncios_total:
    # Cria um DataFrame com os URLs dos anúncios
    df_link = pd.DataFrame({'URL': urls_anuncios_total})
    # Mostra o DataFrame
    print(df_link)

Progresso: 100%|██████████| 1414/1414 [27:25<00:00,  1.16s/it] 

                                                     URL
0      /imovel/lancamento-apartamento-aguas-claras-pe...
1      /imovel/casa-4-quartos-venda-jardim-botanico-b...
2      /imovel/casa-condominio-3-quartos-venda-alto-d...
3      /imovel/casa-4-quartos-venda-asa-norte-brasili...
4      /imovel/lancamento-2-e-3-quartos-padrao-luxo-3...
...                                                  ...
42500  /imovel/apartamento-2-quartos-venda-samambaia-...
42501  /imovel/casa-2-quartos-venda-arniqueira-aguas-...
42502  /imovel/casa-4-quartos-venda-taguatinga-norte-...
42503  /imovel/apartamento-3-quartos-venda-sul-aguas-...
42504  /imovel/apartamento-1-quarto-venda-norte-aguas...

[42505 rows x 1 columns]


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Função para extrair as informações de um anúncio a partir de um URL
def extrair_info_anuncio(url):
    # Faz a requisição HTTP para o URL do anúncio
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai as informações do anúncio
        try:
            titulo = soup.find('h1', class_='mb-0 font-weight-600 mobile-fs-1-5').text.strip()
            preco = soup.find('h6', class_='align-text-left text-normal bb-m mr-6').text.split('\n')[2].replace('.','')
            area = soup.find('h6', class_='text-normal mb-0').text.split()[2].replace('.','').replace(',',".")
            descricao = soup.find('p', class_='w-100 pb-3 mb-0 texto-descricao').text
            try:
               area = float(area)
            except:
               area = None
            try:
               preco = float(valor)
            except:
               valor = None
            try:
               valor_metro = float(area)/float(preco)
            except:
               valor_metro = None
  
            info = soup.find_all("h6", class_='mb-0 text-normal')
            imobiliaria = soup.find('h6', class_='pb-0 mb-0').text
            creci = soup.find('div',class_='col-md-8 mt-2 ml-2').text.split('\n')[2].split()[1]
            tipo = soup.find('div', class_='col-lg-6').text.split()[2]

            possibilidades = [
            'Aquecimento Solar', 'Circuito de TV', 'Gerador de Energia', 'Piscina', 'Sala de Ginástica',
            'Varanda', 'Cozinha Espaçosa', 'Ar Condicionado', 'Cobertura Coletiva', 'Guarita',
            'PlayGround', 'Salão de Festas', 'Projeto de Iluminação', 'Pintura Nova', 'Área de Lazer',
            'Interfone', 'Mobiliado', 'Área de Serviço', 'Brinquedoteca', 'Churrasqueira', 'Cozinha com Armários',
            'Despensa', 'Escritório', 'Gás Canalizado', 'Jardim', 'Lavabo', 'Lazer no Pilotis', 'Poço Artesiano',
            'Portão Eletrônico', 'Quadra Esportiva', 'Sala de Jogos', 'Salão de Jogos', 'Salão Gourmet', 'Sauna',
            'SPA', 'Área de Serviço Coberta', 'Piso em Porcelanato', 'Vista Livre', 'Home Theater', 'Aceita Pet', 'Academia'
            ]

            resultados = {possibilidade: 0 for possibilidade in possibilidades}
            checkbox = soup.find('ul', class_='checkboxes')
            checkbox_text = checkbox.text.strip()
            checkbox_list = checkbox_text.split('\n')

            for item in checkbox_list:
              if item in resultados:
                resultados[item] = 1

            dados_dict = {}
            for i in info:
              item  = i.text.split()
              chave = item[0]
              valor = ' '.join(item[1:])
              if chave in dados_dict:
                dados_dict[chave].append(valor)
              else:
                dados_dict[chave] = [valor]
            try:
              quartos = dados_dict['Quartos:'][0]
            except:
              quartos = 0
            try:
              suites = dados_dict['Suítes:'][0]
            except:
              suites = 0
            try:
              garagem = dados_dict['Garagens:'][0]
            except:
              garagem = 0
            try:
              iptu = dados_dict['IPTU'][0]
            except:
              iptu = '0 SemInformacao'
            try:
              condominio = dados_dict['Condomínio'][0]
            except:
              condominio = '0 SemInformacao'
            try:
              regiao = dados_dict['Cidade:'][0]
            except:
              regiao = '0 SemInformacao'
        except AttributeError as e:
            print(e)
            return None
        # Retorna um dicionário com as informações do anúncio
        return {
            'Título': titulo,
            "Valor": preco,
            "area": area,
            'Valor_metro': valor_metro,
            'quartos': quartos,
            'suites': suites,
            'garagem': garagem,
            'iptu': iptu.split()[1].replace('.',''),
            'condominio': condominio.split()[1].replace('.',''),
            'descricao': descricao, 
            'regiao': regiao.split('-')[1].strip(),
            'cidade': regiao,
            'imobiliaria': imobiliaria,
            'creci': creci,
            "operacao": 'venda',
            **resultados,
            'tipo': tipo
        }
    else:
        print('Falha ao acessar o anúncio:', response.status_code)
        return None

# Lista de URLs dos anúncios
urls_anuncios = [f"https://www.dfimoveis.com.br/{pagina}" for pagina in df_link['URL'].values[0:500]]

# Lista para armazenar as informações dos anúncios
info_anuncios_total = []

# Extrai as informações de cada anúncio
for url_anuncio in tqdm(urls_anuncios, desc='Extraindo informações dos anúncios'):
    info_anuncio = extrair_info_anuncio(url_anuncio)
    if info_anuncio:
        info_anuncios_total.append(info_anuncio)

# Se as informações foram obtidas com sucesso
if info_anuncios_total:
    # Cria um DataFrame com as informações dos anúncios
    df = pd.DataFrame(info_anuncios_total)

df

Extraindo informações dos anúncios:   0%|          | 0/500 [00:00<?, ?it/s]

'NoneType' object has no attribute 'text'


Extraindo informações dos anúncios:   1%|          | 6/500 [00:01<01:46,  4.64it/s]

'NoneType' object has no attribute 'text'


Extraindo informações dos anúncios:   2%|▏         | 9/500 [00:01<01:47,  4.55it/s]

...

'NoneType' object has no attribute 'text'


Extraindo informações dos anúncios:  67%|██████▋   | 337/500 [01:23<01:00,  2.70it/s]

'NoneType' object has no attribute 'text'


Extraindo informações dos anúncios: 100%|██████████| 500/500 [02:11<00:00,  3.79it/s]


,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,descricao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,Alameda Beija flores,1650000,370.00,0.000224,4,4,2,SemInformacao,777,Mikonos House – No Residencial Santa Mônica - ...,...,0,1,1,0,0,1,0,0,0,Casa
1,"RODOVIA BR 0020 KM 12,5",1240000,200.00,0.000161,3,3,2,SemInformacao,SemInformacao,Imobiliária Margarete Corrêa vende Excelente C...,...,0,0,0,0,0,0,0,0,0,Casa
2,SHCGN 713 Bloco Q,2490000,257.00,0.000103,4,3,2,2608,SemInformacao,Kontá Imóveis: Perto de você!\r\rCasa construí...,...,0,0,0,0,0,0,0,0,0,Casa
3,SQSW 100 Bloco B,1390000,97.22,0.000070,3,1,1,2564,1061,Kontá Imóveis: Perto de você!\r\rEXCLUSIVIDADE...,...,0,0,0,0,0,1,0,0,0,Apartamento
4,SHIS QL 8,6900000,750.00,0.000109,5,5,4,SemInformacao,SemInformacao,Piso inferior: Quatro espaçosas salas aguarda...,...,0,1,0,0,0,0,0,0,0,Casa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,Rua 36 - Top Life Club Residence,628000,86.47,0.000138,3,1,2,1023,686,Exclusividade - Bem-vindo ao seu novo lar no c...,...,0,0,0,0,0,0,0,0,0,Apartamento
411,SQN 112 Bloco I,1369000,131.00,0.000096,2,0,1,183,1097,Apartamento de 02 quartos - SQN 112 Bloco I \n...,...,0,0,0,0,0,0,0,0,0,Apartamento
412,Rua 28,1590000,169.00,0.000106,4,3,3,169,1451,"Apartamento de alto padrão, vazado, vista livr...",...,0,0,0,0,0,0,0,0,0,Apartamento
413,QRSW 8 Bloco A 7,675000,61.00,0.000090,2,0,0,99,550,"Praia dos Ingleses - Apartamento vazado, vista...",...,0,0,0,0,0,0,0,0,0,Apartamento


In [18]:
df.to_csv('df_venda.csv', index=False)

In [21]:
df_aluguel = pd.read_csv('aluguel.csv')
df_venda = pd.read_csv('df_vendas.csv')


In [22]:
df_total = pd.concat([df_aluguel,df_venda])

In [23]:
df_total.drop_duplicates()
df_total.to_csv('imoveis.csv', index=False)


In [24]:
import pandas as pd
df = pd.read_csv("imoveis.csv")

In [25]:
df

,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,descricao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,SCS Quadra 06 Bloco A Lote 194,7000.0,200.00,0.028571,0,0,0,SemInformacao,2000,"6º andar todo reformado, com 200 m², piso em p...",...,0,0,0,0,0,0,0,0,0,Prédio
1,SQNW 310 Bloco B,4900.0,76.00,0.015510,2,1,2,409,791,Excelente apartamento no condomínio Persona Ve...,...,0,1,0,0,0,0,0,0,0,Apartamento
2,COLONIA AGRICOLA SAMAMBAIA,9000.0,765.00,0.085000,5,3,21,SemInformacao,SemInformacao,"Colônia Agrícola - Sobrado com 5 dormitórios, ...",...,0,1,0,0,0,0,0,0,0,Casa
3,SBS Quadra 02 Bloco S Lote 01,700.0,28.00,0.040000,0,0,0,257,562,"Excelente sala comercial com piso laminado, ar...",...,0,0,0,0,0,0,0,0,0,Sala
4,Condomínio Prive Morada Sul,7950.0,500.00,0.062893,5,3,4,SemInformacao,849,Sem dúvida esta é uma daquelas oportunidades q...,...,0,0,0,0,0,0,0,0,0,Casa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49723,QS 502,268000.0,51.00,0.000190,2,1,1,424,397,"- 2 quartos, sendo 1 suíte com amários- Cozin...",...,0,0,0,0,0,0,0,0,0,Apartamento
49724,SHA Conjunto 5 Chacará 125,410000.0,130.00,0.000317,2,1,5,SemInformacao,SemInformacao,Beiramar apresenta:Casa em Arniqueira com loca...,...,0,0,0,0,0,0,0,0,0,Casa
49725,QNA 24,650000.0,280.00,0.000431,5,1,4,SemInformacao,SemInformacao,"Ágio casa qna,5 qtos , garagem pra 4 carros es...",...,0,0,0,0,0,0,0,0,0,Casa
49726,Rua COPAIBA,550000.0,84.00,0.000153,3,2,1,SemInformacao,SemInformacao,.,...,0,0,0,0,0,0,0,0,0,Apartamento
